In [1]:
import os
import json
import jsonlines
from collections import defaultdict
import time

In [2]:
all_start = time.time()

In [3]:
# CMeIE
# A. c→(h,r)
out_name = 'c_to_hr_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本中，先抽取可能存在的主体，并基于主体从给定实体列表中提取可能的实体类型。给定的实体类型列表：{entity_list}。
然后基于主体和实体类型从给定关系列表中提取可能的关系。给定的关系列表{relation_list}。
任务的输出形式是（主体||实体类型||关系类型）。
给定文本：“{text}”
'''
oup_prompt = '{answer_text}'

In [4]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [5]:
read_dir = './ori/CMeIE-V2/'
out_dir = './bidirection/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [6]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))
ent_type_list = []
for data in trip_types_list:
    ent_type_list.append(data['subject_type'])
    ent_type_list.append(data['object_type'])
ent_types = sorted(set(ent_type_list),reverse=True)

In [7]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'])
            spo_list = [(item['subject'],item['subject_type'],item['predicate']) for item in data['spo_list']]
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['（{}||{}||{}）'.format(item[0],item[1],item[2]) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_hr_withtype
cost:2.26秒


In [8]:
# CMeIE
# B. c→(t,r)
out_name = 'c_to_tr_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本中，先抽取可能存在的客体，并基于客体从给定实体列表中提取可能的实体类型。给定的实体类型列表：{entity_list}。
然后基于客体和实体类型从给定关系列表中提取可能的关系。给定的关系列表{relation_list}。
任务的输出形式是（客体||实体类型||关系类型）。
给定文本：“{text}”
'''
oup_prompt = '{answer_text}'

In [9]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [10]:
read_dir = './ori/CMeIE-V2/'
out_dir = './bidirection/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [11]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [12]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'])
            spo_list = [(item['object']['@value'],item['object_type']['@value'],item['predicate']) for item in data['spo_list']]
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['（{}||{}||{}）'.format(item[0],item[1],item[2]) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_tr_withtype
cost:2.53秒


In [13]:
# CMeIE
# C. h[s1]c→(t,r)
out_name = 'sc_to_tr_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本、主体和主体类型，先抽取可能存在的客体，并基于客体从给定实体列表中提取可能的实体类型。给定的实体类型列表：{entity_list}。
然后基于主客实体对及其对应的实体类型从给定关系列表中提取可能的关系。给定的关系列表{relation_list}。
任务的输出形式是（客体||实体类型||关系类型）。
给定文本：“文本：{text}；主体：{subject}；主体类型：{subject_type}”
'''
oup_prompt = '{answer_text}'

In [14]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [15]:
read_dir = './ori/CMeIE-V2/'
out_dir = './bidirection/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [16]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [17]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            spo_def_list = defaultdict(list)
            for spo_item in data['spo_list']:
                subject = spo_item['subject']
                subject_type = spo_item['subject_type']
                predicate = spo_item['predicate']
                obj = spo_item['object']['@value']
                obj_type = spo_item['object_type']['@value']
                spo_def_list[(subject,subject_type)].append((obj,obj_type,predicate))
            for sub,sub_type in spo_def_list.keys():
                inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'],subject=sub,subject_type = sub_type)
                spo_list = spo_def_list[(sub,sub_type)]
                oup = '\n'.join(['（{}||{}||{}）'.format(item[0],item[1],item[2]) for item in spo_list])
                oup = '```\n' + oup.strip() + '\n```'
                out_data = {
                    'instruction':inp,
                    'input':'',
                    'output':oup,
                    'text':data['text'],
                    'spo_list':data['spo_list']
                }
                fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:sc_to_tr_withtype
cost:2.54秒


In [18]:
# CMeIE
# D. t[s1]c→(h,r). 
out_name = 'sc_to_hr_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本、客体和客体类型，先抽取可能存在的主体，并基于主体从给定实体列表中提取可能的实体类型。给定的实体类型列表：{entity_list}。
然后基于主客实体对及其对应的实体类型从给定关系列表中提取可能的关系。给定的关系列表{relation_list}。
任务的输出形式是（主体||实体类型||关系类型）。
给定文本：“文本：{text}；客体：{object}；客体类型：{object_type}”
'''
oup_prompt = '{answer_text}'

In [19]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [20]:
read_dir = './ori/CMeIE-V2/'
out_dir = './bidirection/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [21]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [22]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            spo_def_list = defaultdict(list)
            for spo_item in data['spo_list']:
                subject = spo_item['subject']
                subject_type = spo_item['subject_type']
                predicate = spo_item['predicate']
                obj = spo_item['object']['@value']
                obj_type = spo_item['object_type']['@value']
                spo_def_list[(obj,obj_type)].append((subject,subject_type,predicate))
            for obj,obj_type in spo_def_list.keys():
                inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'],object=obj,object_type = obj_type)
                spo_list = spo_def_list[(obj,obj_type)]
                oup = '\n'.join(['（{}||{}||{}）'.format(item[0],item[1],item[2]) for item in spo_list])
                oup = '```\n' + oup.strip() + '\n```'
                out_data = {
                    'instruction':inp,
                    'input':'',
                    'output':oup,
                    'text':data['text'],
                    'spo_list':data['spo_list']
                }
                fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:sc_to_hr_withtype
cost:3.16秒


In [23]:
# CMeIE
# A0. c→(h,r,t)
out_name = 'c_to_hrt_withtype'
inp_prompt = '''当前你是一个资深的信息提取的专家。
你的任务是从给定文本中，抽取关系三元组。先从给定文本中抽取主客体实体对，并基于主客体实体对从给定实体列表中提取主客体的实体类型。给定的实体类型列表：{entity_list}。
然后基于主客实体对及其对应的实体类型从给定关系列表中提取可能的关系。给定的关系列表：{relation_list}。
任务的输出形式是（主体||主体类型||关系类型||客体||客体类型）。
给定文本：“{text}”
'''
oup_prompt = '{answer_text}'

In [24]:
data_file_list = [
    'CMeIE-V2_train.jsonl',
    'CMeIE-V2_dev.jsonl',
    'CMeIE-V2_test.jsonl',
]
out_file_list = [
    'train.json',
    'dev.json',
    'test.json'
]

In [25]:
read_dir = './ori/CMeIE-V2/'
out_dir = './bidirection/CMeIE-V2'
out_dir = os.path.join(out_dir,out_name)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
with open(os.path.join(read_dir,'53_schemas.json'),'r') as f:
    trip_types_list = json.load(f)

In [26]:
trip_types = sorted(set([data['predicate'] for data in trip_types_list]))

In [27]:
start = time.time()
print('out_name:{}'.format(out_name))
for data_file,out_file in zip(data_file_list,out_file_list):
    read_path = os.path.join(read_dir,data_file)
    out_path = os.path.join(out_dir,out_file)
    with jsonlines.open(read_path,'r') as f:
        datas = [data for data in f]
    with jsonlines.open(out_path,'w') as fw:
        for data in datas:
            inp = inp_prompt.format(relation_list='，'.join(trip_types),entity_list = '，'.join(ent_types),text=data['text'])
            spo_list = [(item['subject'],item['subject_type'],item['predicate'],item['object']['@value'],item['object_type']['@value']) for item in data['spo_list']]
            processed_spo_list = []
            for spo_item in spo_list:
                if spo_item not in processed_spo_list:
                    processed_spo_list.append(spo_item)
            oup = '\n'.join(['（{}||{}||{}||{}||{}）'.format(item[0],item[1],item[2],item[3],item[4]) for item in processed_spo_list])
            oup = '```\n' + oup.strip() + '\n```'
            out_data = {
                'instruction':inp,
                'input':'',
                'output':oup,
                'text':data['text'],
                'spo_list':data['spo_list']
            }
            fw.write(out_data)
end = time.time()
print('cost:{}秒'.format(round(end-start, 2)))

out_name:c_to_hrt_withtype
cost:1.7秒


In [28]:
all_end = time.time()
all_end - all_start

24.777007341384888